In [1]:
import numpy as np
import cv2 as cv
import time
from numba import njit

## with opencv

In [14]:
start = time.time()
cap = cv.VideoCapture('Narrow_Neck_BW_1.mov')
fl_to_show=1
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 20.0, (1440,  810),False)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv.resize(frame, (1440, 810))
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#     ret_prep,thresh = cv.threshold(frame,127,255,cv.THRESH_BINARY)
#     ret_prep,thresh = cv.threshold(frame,127,255,cv.THRESH_BINARY_INV)
    thresh = cv.adaptiveThreshold(frame,255,cv.ADAPTIVE_THRESH_MEAN_C,\
                                  cv.THRESH_BINARY,11,2)
    out.write(thresh)
#     thresh = cv.adaptiveThreshold(frame,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv.THRESH_BINARY,11,2)
    if cv.waitKey(1) == ord('2'):
        if fl_to_show==1: fl_to_show = 2
        else: fl_to_show = 1
    if fl_to_show==1:
        frame_to_show = frame   
    else: 
        frame_to_show = thresh
        
    cv.imshow('frame', frame_to_show)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
out.release()
cv.destroyAllWindows()
end = time.time()
print(round((end-start),2))

14.19


##  native python

In [6]:
def adaptive_binarization_mean(frame,blockSize, constant):
    height = np.shape(frame)[0]//blockSize
    width = np.shape(frame)[1]//blockSize
    frame_binary = frame
    for h in range(height):
        for w in range(width):
            s = 0
            for i in range(blockSize):
                for j in range(blockSize):
                    s = s + frame[h*blockSize+i][w*blockSize+j]
            threshhold = s/(blockSize**2)
            for i in range(blockSize):
                for j in range(blockSize):
                    if frame_binary[h*blockSize+i][w*blockSize+j]>threshhold:
                        frame_binary[h*blockSize+i][w*blockSize+j]=255
                    else: frame_binary[h*blockSize+i][w*blockSize+j] = 0
                   
    return frame_binary

In [7]:
start = time.time()
cap = cv.VideoCapture('Narrow_Neck_BW_1.mov')
fl_to_show=1
fourcc = cv.VideoWriter_fourcc(*'XVID')
height = 810
width = 1440
blocksize = 11
constant = 2
out = cv.VideoWriter('output.avi', fourcc, 20.0, (width,  height),False)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame2 = frame
    frame = cv.resize(frame, (width//blocksize*blocksize, height//blocksize*blocksize))

    
    thresh = adaptive_binarization_mean(frame2,blocksize,constant)
    thresh = cv.resize(thresh, (width, height))
    frame = cv.resize(frame, (width, height))
    out.write(thresh)
    if cv.waitKey(25) == ord('2'):
        if fl_to_show==1: fl_to_show = 2
        else: fl_to_show = 1
    if fl_to_show==1:
        frame_to_show = frame   
    else: 
        frame_to_show = thresh
        
    cv.imshow('frame', frame_to_show)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
end = time.time()
print(round((end-start),2))

17.12


## with numba

In [15]:
@njit
def adaptive_binarization_mean(frame,blockSize, constant):
    height = np.shape(frame)[0]//blockSize
    width = np.shape(frame)[1]//blockSize
    frame_binary = frame
    for h in range(height):
        for w in range(width):
            s = 0
            for i in range(blockSize):
                for j in range(blockSize):
                    s = s + frame[h*blockSize+i][w*blockSize+j]
            threshhold = s/(blockSize**2)
            for i in range(blockSize):
                for j in range(blockSize):
                    if frame_binary[h*blockSize+i][w*blockSize+j]>threshhold:
                        frame_binary[h*blockSize+i][w*blockSize+j]=255
                    else: frame_binary[h*blockSize+i][w*blockSize+j] = 0
                   
    return frame_binary

In [16]:
start = time.time()
cap = cv.VideoCapture('Narrow_Neck_BW_1.mov')
fl_to_show=1
fourcc = cv.VideoWriter_fourcc(*'XVID')
height = 810
width = 1440
blocksize = 11
constant = 2
out = cv.VideoWriter('output.avi', fourcc, 20.0, (width,  height),False)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame2 = frame
    frame = cv.resize(frame, (width//blocksize*blocksize, height//blocksize*blocksize))

    
    thresh = adaptive_binarization_mean(frame2,blocksize,constant)
    thresh = cv.resize(thresh, (width, height))
    frame = cv.resize(frame, (width, height))
    out.write(thresh)
    if cv.waitKey(25) == ord('2'):
        if fl_to_show==1: fl_to_show = 2
        else: fl_to_show = 1
    if fl_to_show==1:
        frame_to_show = frame   
    else: 
        frame_to_show = thresh
        
    cv.imshow('frame', frame_to_show)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
end = time.time()
print(round((end-start),2))

19.9
